In [ ]:
# Reproduction of Figure 5C analysis of spike time response
# This file loads pre-sorted spikes and conducts a windowed analysis of spike rate responses after stimulation

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from pynwb import NWBHDF5IO, NWBFile

### File finder by extension

In [2]:
def find_extension(directory, extension='.npy'):
    matching_files = []
    for file in os.listdir(directory):
        if file.endswith(extension):
                matching_files.append(os.path.join(directory, file))
    matching_files.sort()
    return matching_files

spike_path='../data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117'
file_extension = '.npy'

### Excel file

In [3]:
import pandas as pd
freq = 30000
duration=5
def spike_number_5(matching_files, time, duration=5, freq=30000):  # This function calculates the number of spikes 5 seconds before and after a given time
    data=[]
    if time-duration<0:
        print('The length before and after are not the same!')
    for file in matching_files:
        spike_data=np.load(file,allow_pickle=True)
        spike_data=spike_data/freq
        data_size=spike_data.shape[0]
        for j in range(data_size):
            tmp1=sum((time-duration<=spike_data[j]) & (spike_data[j]<time))
            tmp2=sum((time<=spike_data[j]) & (spike_data[j]<time+duration))
            before=tmp1
            after=tmp2
        # print(f'before {time} : {sum(before)}, after {time} : {sum(after)}')
            data.append({
                    'name': os.path.basename(os.path.dirname(os.path.dirname(file))),
                    'day': os.path.splitext(os.path.basename(file))[0],
                    'neuron': f'# {j+1}',
                    f'before_{time}': int(before),
                    f'after_{time}': int(after),
                })
    df = pd.DataFrame(data, columns=['name', 'day', 'neuron', f'before_{time}', f'after_{time}'])
    return df
# spike_day
matching_files = find_extension(spike_path, file_extension)
matching_files
df=spike_number_5(matching_files,30)
for time in [45,60]:
    df2=spike_number_5(matching_files, time)
    df=df.join(df2.set_index(['name', 'day', 'neuron'])[[f'before_{time}',f'after_{time}']], on=['name', 'day', 'neuron'], rsuffix=f'_{time}')
output_file = 'spike_data_analysis.xlsx'
df.to_excel(output_file, index=False)
df
# df = pd.read_excel('Updates/number of spikes.xlsx')

,name,day,neuron,before_30,after_30,before_45,after_45,before_60,after_60
0,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 1,5,16,3,18,3,17
1,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 2,3,15,4,19,0,15
2,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 1,12,69,9,73,10,74
3,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 2,3,33,3,31,0,37
4,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 1,10,22,8,31,1,19
5,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 2,11,36,9,34,5,40
6,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 1,13,97,12,50,1,55
7,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 2,2,43,6,18,0,20
8,spikingdata_v1,BO2_231117_Stim50uAspikes_v0,# 1,0,5,3,7,0,1
9,spikingdata_v1,BO2_231117_Stim5uAspikes_v0,# 1,1,3,5,7,2,5


### Calculating scan statistics for each of the neurons

In [4]:
from scipy import stats
from scipy.stats import anderson

df['p_value']=None
for i in range(df.shape[0]):
    dif=[0,0,0]
    if not pd.isna(df.iloc[i][2]):
        dif[0]=df.iloc[i][4]-df.iloc[i][3]
        dif[1]=df.iloc[i][6]-df.iloc[i][5]
        dif[2]=df.iloc[i][8]-df.iloc[i][7]
        dif=np.array(dif)
        firings_before=[df.iloc[i][3],df.iloc[i][5],df.iloc[i][7]]
        firings_after=[df.iloc[i][4],df.iloc[i][6],df.iloc[i][8]]
        stat, p_value = stats.shapiro(dif)
        # t_stat, p_value = stats.ttest_rel(firings_after, firings_before)
        # t_statistic, p_value = stats.ttest_1samp(dif, 0)
        result = anderson(dif, dist='norm')
        # print(result.critical_values)
        print(dif, f'p-value: {p_value}')
        # if p_value<0.05:
        df['p_value'][i]=p_value
df

[11 15 14] p-value: 0.46326287493379903
[12 15 15] p-value: 7.4208040309766365e-16
[57 64 64] p-value: 7.4208040309766365e-16
[30 28 37] p-value: 0.4072116279909085
[12 23 18] p-value: 0.8998502800372316
[25 25 35] p-value: 0.0
[84 38 54] p-value: 0.6677997841574009
[41 12 20] p-value: 0.5163183143604517
[5 4 1] p-value: 0.46326287493379903
[2 2 3] p-value: 0.0
[ 21 -16  24] p-value: 0.1286869740943084
[ 5 10  0] p-value: 1.0
[ 5  5 -3] p-value: 7.4208040309766365e-16


/var/folders/n9/nrf9s6h16xs3pbl0jmsw7lnc0000gq/T/ipykernel_39264/1328135751.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['p_value'][i]=p_value


,name,day,neuron,before_30,after_30,before_45,after_45,before_60,after_60,p_value
0,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 1,5,16,3,18,3,17,0.463263
1,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 2,3,15,4,19,0,15,0.0
2,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 1,12,69,9,73,10,74,0.0
3,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 2,3,33,3,31,0,37,0.407212
4,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 1,10,22,8,31,1,19,0.89985
5,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 2,11,36,9,34,5,40,0.0
6,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 1,13,97,12,50,1,55,0.6678
7,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 2,2,43,6,18,0,20,0.516318
8,spikingdata_v1,BO2_231117_Stim50uAspikes_v0,# 1,0,5,3,7,0,1,0.463263
9,spikingdata_v1,BO2_231117_Stim5uAspikes_v0,# 1,1,3,5,7,2,5,0.0


In [48]:
df.to_excel('Updates.xlsx', index=False)

### Scan statistics

In [5]:
def b_k(k,n,w):
    return stats.binom.pmf(k,n,w)

def g_b(k,n,w):
    g=0
    for i in range(k,n+1):
        g+=b_k(i,n,w)
    return g

def scan_prob(k,n,w=0.5):
    return (k/w-n-1)*b_k(k,n,w)+2*g_b(k,n,w)

# scan_prob(7,100,0.125)
for i in range(len(df)):
    n_30 = df.iloc[i, 3] + df.iloc[i, 4]
    k_30 = df.iloc[i, 4]
    df.at[i, 'scan_statistic_30'] = scan_prob(k_30, n_30, 0.5)

    n_45 = df.iloc[i, 5] + df.iloc[i, 6]
    k_45 = df.iloc[i, 6]
    df.at[i, 'scan_statistic_45'] = scan_prob(k_45, n_45, 0.5)

    n_60 = df.iloc[i, 7] + df.iloc[i, 8]
    k_60 = df.iloc[i, 8]
    df.at[i, 'scan_statistic_60'] = scan_prob(k_60, n_60, 0.5)
df

,name,day,neuron,before_30,after_30,before_45,after_45,before_60,after_60,p_value,scan_statistic_30,scan_statistic_45,scan_statistic_60
0,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 1,5,16,3,18,3,17,0.463263,1.236353e-01,1.036835e-02,1.671028e-02
1,spikingdata_v1,BO2_231117_Stim10uAspikes_v0,# 2,3,15,4,19,0,15,0.0,4.177856e-02,1.737785e-02,4.882812e-04
2,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 1,12,69,9,73,10,74,0.0,1.708380e-09,3.955584e-12,9.321296e-12
3,spikingdata_v1,BO2_231117_Stim20uAspikes_v0,# 2,3,33,3,31,0,37,0.407212,3.240333e-06,1.017051e-05,2.764864e-10
4,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 1,10,22,8,31,1,19,0.89985,2.153271e-01,2.756120e-03,3.643036e-04
5,spikingdata_v1,BO2_231117_Stim30uAspikes_v0,# 2,11,36,9,34,5,40,0.0,3.316149e-03,1.708807e-03,1.259416e-06
6,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 1,13,97,12,50,1,55,0.6678,1.742415e-15,1.854550e-05,4.277134e-14
7,spikingdata_v1,BO2_231117_Stim40uAspikes_v1,# 2,2,43,6,18,0,20,0.516318,1.184389e-09,1.109039e-01,2.002716e-05
8,spikingdata_v1,BO2_231117_Stim50uAspikes_v0,# 1,0,5,3,7,0,1,0.463263,1.875000e-01,6.953125e-01,1.000000e+00
9,spikingdata_v1,BO2_231117_Stim5uAspikes_v0,# 1,1,3,5,7,2,5,0.0,8.750000e-01,9.677734e-01,7.812500e-01


### Raster plot and intera-spike interval (ISI) plot and histogram

In [8]:
def event_raster_plot(matching_files,event=True,raster=True):
    data_size=[]
    for file in matching_files:
        spike_data=np.load(file,allow_pickle=True)
        data_size.append(spike_data.shape[0])
        if event==True:
            plt.figure(figsize=(7*data_size[-1], 7*data_size[-1]))
            for i in range(data_size[-1]):
                diff = np.diff(spike_data[i])
                plt.subplot(data_size[-1], 2, i*2+1)  
                plt.plot(diff, marker='o', linestyle='-')  
                plt.xlabel('Events(first difference)')  
                plt.ylabel('Time (ms)')  
                plt.title('Firing Plot')
                plt.subplot(data_size[-1], 2, i*2+2)
                plt.hist(diff, bins=20) 
                plt.grid(True)
                plt.tight_layout()
            plt.savefig(os.path.splitext(os.path.basename(file))[0]+'.png')
            plt.close()
        if raster==True:
            for i in range(data_size[-1]):
                neuron_id=i+1
                time_of_spike=spike_data[i]
                plt.plot(time_of_spike,neuron_id*np.ones(len(spike_data[i])), marker='*', linestyle='-')  
                plt.xlabel('Time of fire (ms)')  
                plt.ylabel('Neuron ID')  
                plt.title('Raster Plot')
            plt.savefig(os.path.splitext(os.path.basename(file))[0]+'_raster.png')
            plt.close()

matching_files = find_extension(spike_path, file_extension)
# event_raster_plot(matching_files)
len(matching_files)
matching_files
# matching_files.sort()


['./GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim10uAspikes_v0.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim20uAspikes_v0.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim30uAspikes_v0.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim40uAspikes_v1.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim50uAspikes_v0.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim5uAspikes_v0.npy',
 './GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim60uAspikes_v0.npy']

### Stimulation times for Experimental_Data_2month_Round_1

These are the time points provided by the experimenter for the stimulation times in the Experimental_Data_2month_Round_1 dataset.

In [ ]:
ss_time=[[601.211933, 603.043],[601.090133,602.9212],[600.988800,602.819867],[600.951700,602.782767],[422.739600,424.570567]]
ss_time=np.array(ss_time)

### approximate stimulations times for Organoid Batch 14 Set ~ 30,45,60 seconds

### General function to generate the Raster plot, the intera-spike interval (ISI) plot and histogram, cumulative time of spike firing and moving average of spike firings.

In [12]:
def time_event_raster_window(matching_files, stim_time,window_size=120,time=True, event=False, raster=False, window=False):
    data_size=[]
    for ff in range(len(matching_files)):
        file=matching_files[ff]
        # start_stim=stim_time[ff][0]
        # stop_stim=stim_time[ff][1]
        spike_data=np.load(file,allow_pickle=True)/30000 # convert to seconds
        data_size.append(spike_data.shape[0])
        fig1 = plt.figure(figsize=(7*data_size[-1], 7*data_size[-1]))
        fig2 = plt.figure(figsize=(7*data_size[-1], 7*data_size[-1]))
        fig3 = plt.figure(figsize=(7*data_size[-1], 7*data_size[-1]))
        fig4 = plt.figure(figsize=(7*data_size[-1], 7*data_size[-1]))
        ax3 = fig3.add_subplot(1,1,1)
        ax4 = fig4.add_subplot(1,1,1)
        legend_labels=[]
        for i in range(data_size[-1]):
            window_size_0=window_size
            moving_window=[]
            neuron_id=i+1
            org = spike_data[i]
            diff = np.diff(spike_data[i])
            ax1 = fig1.add_subplot(data_size[-1], 2, i*2+1)
            ax1_2 = fig1.add_subplot(data_size[-1], 2, i*2+2)
            ax1.plot(diff, marker='o', linestyle='-')  
            ax1.set_xlabel('Events(first difference)')  
            ax1.set_ylabel('Time (s)')  
            ax1.set_title('Overall activity Plot')
            ax1_2.hist(diff, bins=20) 
            ax1.grid(True)
            ax2 = fig2.add_subplot(data_size[-1], 1, i+1)
            ax2.plot(org, marker='o', linestyle='-')  
            ax2.set_xlabel('Events',fontsize=26)  
            ax2.set_ylabel('Cumulative Time (s)',fontsize=26)  
            ax2.set_title('Firing Plot',fontsize=26)
            ax2.grid(True)
            recording_time = int(np.max([np.max(inner_array) for inner_array in spike_data]))+1
            overall_window=recording_time-window_size_0+1
            ax3.plot(spike_data[i],neuron_id*np.ones(len(spike_data[i])), marker='*', markersize=25,linewidth=2.5,linestyle='-')  
            ax3.set_xlabel('Time of fire (s)',fontsize=25*data_size[-1])  
            ax3.set_ylabel('Neuron ID',fontsize=25*data_size[-1])  
            ax3.set_title('Raster Plot',fontsize=25*data_size[-1])
            ## ax3.axvline(x=int(recording_time/2), color='r', linewidth=4, linestyle=':')
            # ax3.axvspan(start_stim, stop_stim, color='gray', alpha=0.5)
            ax3.tick_params(axis='x', labelsize=15*data_size[-1])
            ax3.tick_params(axis='y', labelsize=15*data_size[-1])
            if overall_window<0:
                window_size_0=recording_time
                overall_window=recording_time-window_size_0+1
            for j in range (0, overall_window):
                _moving_window=sum((j<=spike_data[i]) & (spike_data[i]<=j+window_size_0))
                moving_window.append(_moving_window)
            legend_labels.append(f'{neuron_id}')
            ax4.plot(np.arange(overall_window),moving_window, marker='*', markersize=25,linewidth=2.5,linestyle='-')
            ax4.set_xlabel('Time window(s)',fontsize=25*data_size[-1])  
            ax4.set_ylabel('Number of spikes',fontsize=25*data_size[-1])
            ax4.set_title(f'Window size: {window_size_0} (s)',fontsize=25*data_size[-1])
            if window_size_0<window_size:
                ax4.set_title(f'Window size: {window_size_0} (s) !!', fontsize=25*data_size[-1]) 
            ax4.tick_params(axis='x', labelsize=15*data_size[-1])
            ax4.tick_params(axis='y', labelsize=15*data_size[-1])
            ax4.grid(True)
        ax4.legend(legend_labels,prop={'size': 44})
        try:
            # x_stim = int(recording_time/2-window_size+1)
            # ax4.axvline(x=x_stim, color='r', linestyle='--', linewidth=4)
            # x_stim = int(start_stim)-window_size+1
            # ax4.axvline(x=x_stim, color='r', linestyle='--', linewidth=4)
            ax4.axvline(x=20, color='k', linestyle='--', linewidth=4)
            ax4.axvline(x=35, color='k', linestyle='--', linewidth=4)
            ax4.axvline(x=50, color='k', linestyle='--', linewidth=4)
        except:
            pass
        if time==True:
            fig1.savefig(os.path.splitext(os.path.basename(file))[0]+'.png')
        plt.close(fig1)
        if event==True:
            fig2.savefig(os.path.splitext(os.path.basename(file))[0]+'_org.png')
        plt.close(fig2)
        if raster==True:
            fig3.savefig(os.path.splitext(os.path.basename(file))[0]+'_raster.png')
        plt.close(fig3)
        if window==True:
            fig4.savefig(os.path.splitext(os.path.basename(file))[0]+'_window.png')
        plt.close(fig4)
            
T=True
F=False
# ss_time=1
window_size=10
time_event_raster_window(matching_files,ss_time ,window_size=window_size, time=F, event=F, raster=T, window=T)

### Moving average of spike rate for the entire organoid culture. (sum of all the units in one graph)

In [13]:
def org_spike(matching_files,stim_time,window_size=120,freq=30000):
    def sum_interval_elements(array, freq=freq):
        sums = []
        for i in range(0, len(array), freq):
            # Sum every three elements and append to the sums list
            sums.append(sum(array[i:i+freq]))
        return sums
    for ff in range(len(matching_files)):
        file=matching_files[ff]
        # start_stim=stim_time[ff][0]
        # stop_stim=stim_time[ff][1]
        moving_window=[]
        spike_data=np.load(file,allow_pickle=True)
        data_size=spike_data.shape[0]
        fig1 = plt.figure(figsize=(30, 30))
        ax1 = fig1.add_subplot(1,1,1)
        recording_time = (np.max([np.max(inner_array) for inner_array in spike_data]))
        organoid_spikes = np.zeros(recording_time)
        for j in range(data_size):
            for i in range(len(spike_data[j])):
                organoid_spikes[spike_data[j][i]-1]= 1
                
        spikes_in_time= sum_interval_elements(organoid_spikes)
        # first_half = int(sum(spikes_in_time[:int(len(spikes_in_time)/2)]))
        # second_half = int(sum(spikes_in_time[int(len(spikes_in_time)/2):]))
        recording_time_sec=int(recording_time/freq)+1
        overall_window=recording_time_sec-window_size+1
        # if start_stim>recording_time_sec-stop_stim:
        #     second_half = int(sum(spikes_in_time[int(stop_stim):]))
        #     first_half = int(sum(spikes_in_time[int(start_stim+stop_stim)-recording_time_sec:int(start_stim)]))
        # else:
        #     first_half = int(sum(spikes_in_time[:int(start_stim)]))
        #     second_half = int(sum(spikes_in_time[int(stop_stim):int(start_stim+stop_stim)]))
        if overall_window<0:
                window_size_0=recording_time
                overall_window=recording_time-window_size_0+1
        for wnd in range(overall_window):
            _moving_window=sum(spikes_in_time[wnd:wnd+window_size])
            moving_window.append(_moving_window)
        ax1.plot(np.arange(0,overall_window),moving_window,marker='*', markersize=25,linewidth=2.5,linestyle='-')
        try:
            ## x_stim = int(recording_time_sec/2-window_size+1)
            # x_stim_start = start_stim-window_size+1
            # x_stim_stop= stop_stim-window_size+1
            # ax1.axvline(x=x_stim, color='r', linestyle='--', linewidth=4)
            # ax1.axvspan(x_stim_start, x_stim_stop, color='red', alpha=0.2)
            # ax1.text(x_stim-(x_stim/window_size*40), min(moving_window), f'#{first_half}', fontsize=80, color='m')
            # ax1.text(x_stim_start-(x_stim_start/window_size*40), min(moving_window), f'#{first_half}', fontsize=80, color='m')
            # ax1.text(x_stim_start, min(moving_window), f'#{second_half}', fontsize=80, color='m')
            ax1.axvline(x=20, color='k', linestyle='--', linewidth=6)
            ax1.axvline(x=35, color='k', linestyle='--', linewidth=6)
            ax1.axvline(x=50, color='k', linestyle='--', linewidth=6)
        except:
            pass
        ax1.set_xlabel('Time (s)',fontsize=88)
        ax1.set_ylabel('# Spikes between every seconds',fontsize=88)
        ax1.set_title(f'Window size: {window_size} (s)',fontsize=88)
        ax1.tick_params(axis='x', labelsize=66)
        ax1.tick_params(axis='y', labelsize=66)
        ax1.grid(True)
        fig1.savefig(os.path.splitext(os.path.basename(file))[0]+'_organoid_window.png')
        plt.close(fig1)
org_spike(matching_files,ss_time,window_size=window_size,freq=30000)


### Generating all the plots and histograms for the entire organoid culture. (in one go)

In [11]:
org_spike(matching_files,ss_time,window_size=120,freq=30000)
time_event_raster_window(matching_files, ss_time,window_size=120, time=False, event=False, raster=True, window=True)

### Calculating the number of spikes 5 seconds before and after the stimulation, here I generate a sequence number of spikes every 0.5 seconds for 5 seconds before and after the stimulation.

In [18]:
freq = 30000
duration=5
def spike_date(matching_files, duration=5, freq=30000,stims=[30,45,60]):
    '''
    results is a numpy array with the following structure:
    results[day][number of neurons][number of stimulations][time, number of spikes in that time]
    '''
    spike_day=[]
    for file in matching_files:
        spike_data=np.load(file,allow_pickle=True)
        spike_data=spike_data/freq
        data_size=spike_data.shape[0]
        spikes_in_half_sec= []
        for j in range(data_size):
            tmp1_1,tmp2_1,tmp3_1=[],[],[]
            tmp1_2,tmp2_2,tmp3_2=[],[],[]
            for i in reversed(range(1,duration*2+1)):
                tmp1_1.append((stims[0]-(i-1)/2,sum((stims[0]-i/2<=spike_data[j]) & (spike_data[j]<stims[0]-(i-1)/2))))
                tmp2_1.append((stims[1]-(i-1)/2,sum((stims[1]-i/2<=spike_data[j]) & (spike_data[j]<stims[1]-(i-1)/2))))
                tmp3_1.append((stims[2]-(i-1)/2,sum((stims[2]-i/2<=spike_data[j]) & (spike_data[j]<stims[2]-(i-1)/2))))
            for i in reversed(range(1,duration*2+1)):
                tmp1_2.append((stims[0]+duration-(i-1)/2,sum((stims[0]+duration-i/2<=spike_data[j]) & (spike_data[j]<stims[0]+duration-(i-1)/2))))
                tmp2_2.append((stims[1]+duration-(i-1)/2,sum((stims[1]+duration-i/2<=spike_data[j]) & (spike_data[j]<stims[1]+duration-(i-1)/2))))
                tmp3_2.append((stims[2]+duration-(i-1)/2,sum((stims[2]+duration-i/2<=spike_data[j]) & (spike_data[j]<stims[2]+duration-(i-1)/2))))
            spikes_in_half_sec.append(([tmp1_1, tmp1_2], [tmp2_1, tmp2_2], [tmp3_1, tmp3_2]))
        spike_day.append(spikes_in_half_sec)
    return (spike_day)
# spike_day

modeling_data_linear=spike_date(matching_files)

### Calculating the number of spikes for 5 seconds before and after a time point (here interesting time points are the stimulation times 30,45,60 seconds)

In [20]:
freq = 30000
duration=5
def spike_number(matching_files, time, duration=5, freq=30000): 
    spike_day=[]
    if time-duration<0:
        print('The length before and after are not the same!')
    for file in matching_files:
        spike_data=np.load(file,allow_pickle=True)
        spike_data=spike_data/freq
        data_size=spike_data.shape[0]
        spikes_in_neuron= []
        for j in range(data_size):
            tmp1=np.zeros(duration)
            tmp2=np.zeros(duration)
            for i in range(duration):
                tmp1[i]=sum((time-duration+i<=spike_data[j]) & (spike_data[j]<time-duration+i+1))
                tmp2[i]=sum((time+i<=spike_data[j]) & (spike_data[j]<time+i+1))
            spikes_in_neuron.append(([tmp1, tmp2]))
        spike_day.append(spikes_in_neuron)
    return spike_day
# spike_day

spike_number(matching_files,30,duration=5)

[[[array([0., 0., 4., 1., 0.]), array([13.,  0.,  0.,  0.,  3.])],
  [array([0., 0., 2., 0., 1.]), array([14.,  1.,  0.,  0.,  0.])]],
 [[array([0., 3., 5., 2., 2.]), array([68.,  1.,  0.,  0.,  0.])],
  [array([1., 1., 1., 0., 0.]), array([33.,  0.,  0.,  0.,  0.])]],
 [[array([1., 4., 0., 1., 4.]), array([20.,  0.,  0.,  0.,  2.])],
  [array([0., 7., 1., 0., 3.]), array([33.,  0.,  0.,  0.,  3.])]],
 [[array([6., 1., 1., 5., 0.]), array([93.,  1.,  3.,  0.,  0.])],
  [array([0., 0., 2., 0., 0.]), array([42.,  0.,  1.,  0.,  0.])]],
 [[array([0., 0., 0., 0., 0.]), array([0., 0., 0., 5., 0.])]],
 [[array([0., 0., 0., 0., 1.]), array([0., 3., 0., 0., 0.])],
  [array([0., 0., 1., 0., 0.]), array([11., 11.,  0.,  0.,  0.])]],
 [[array([0., 0., 0., 0., 0.]), array([5., 0., 0., 0., 0.])],
  [array([0., 0., 0., 0., 0.]), array([5., 0., 0., 0., 0.])]]]

In [10]:
len(spike_number(matching_files,30))

6

### Histogram of before and after stimulation of inter-arrival time

In [10]:
def hiat(matching_files,time,duration=5,freq=30000):
    spike_day=[]
    for file in matching_files:
        spike_data=np.load(file,allow_pickle=True)
        spike_data=spike_data/freq
        data_size=spike_data.shape[0]
        spikes_in_neuron= []
        for j in range(data_size):
            tmp1=spike_data[j][(time-duration<=spike_data[j]) & (spike_data[j]<time)]
            tmp2=spike_data[j][(time<=spike_data[j]) & (spike_data[j]<time+duration)]
            spikes_in_neuron.append(([tmp1, tmp2]))
        spike_day.append(spikes_in_neuron)
    return spike_day

In [25]:
matching_files[0]

'./GraciasLab_Data/Organoid Batch 14 Set/spikingdata_v1/BO2_231117/BO2_231117_Stim5uAspikes_v0.npy'

In [11]:
stim_times=[30,45,60]
bin_width = (65 - 25) / 50
bin_edges = np.arange(25, 65, bin_width)
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
color=['r','g','b','k','m']
for st in stim_times:
    hist_data = hiat(matching_files,st)
    for i in range(len(hist_data)):
        for j in range(len(hist_data[i])):
            ax1.hist(hist_data[i][j][0],color=color[j], label=f'f:{i},n:{j}', bins=bin_edges)
            ax2.hist(hist_data[i][j][1],color=color[j], label=f'f:{i},n:{j}', bins=bin_edges)
            
ax1.set_title("Before stimulation")
ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Counts")
ax2.set_title("After stimulation")
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Counts")
ax1.set_xlim([25, 60])
ax2.set_xlim([30, 65])
fig1.savefig(os.path.splitext(os.path.basename(matching_files[0]))[0]+'hist_before.png')
plt.close(fig1)
fig2.savefig(os.path.splitext(os.path.basename(matching_files[0]))[0]+'hist_after.png')
plt.close(fig2)